# Graphical User Interface

In [1]:
#importing libraries
from tkinter import *
from tkinter.ttk import *
import pandas as pd
import tkintermapview
from geopy.geocoders import Nominatim
import geocoder
import requests 
import numpy as np
import random 
import matplotlib.pyplot as plt
import joblib
import seaborn as sns
from sklearn.cluster import KMeans 
import folium # plotting library
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import warnings
import time
warnings.filterwarnings('ignore')
%run assist.ipynb
%run api_calls.ipynb
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

In [2]:
#importing various datasets
df = pd.read_excel(r'C:/Users/hp/dissertation/Datasets/top25.xlsx')
df_post = pd.read_excel(r'C:/Users/hp/dissertation/Datasets/area_codes.xlsx')
df_model = pd.read_csv(r'C:/Users/hp/dissertation/Datasets/Indices-2022-05_ONS.csv')
df_land = pd.read_excel(r'C:/Users/hp/dissertation/Datasets/UK-HPI-full-file-2022-05_land_registry_1.xlsx')
df_reg = pd.read_excel(r'C:/Users/hp/dissertation/Datasets/pp-2022-modified.xlsx')
df_price = pd.read_excel(r'C:/Users/hp/dissertation/Datasets/price_post_coordinates.xlsx')
df_dummy = pd.read_csv(r'C:/Users/hp/dissertation/Datasets/dummy_csv.csv')
df_dummy.drop(['Unnamed: 0','Price'],axis=1,inplace=True)
df_city = pd.read_csv(r'C:/Users/hp/dissertation/Datasets/city.csv')

In [ ]:
### creating window for gui
window = Tk()
window.title("House Price Prediction System")
width = window.winfo_screenwidth()
height = window.winfo_screenheight()

# setting tkinter window size
window.geometry("%dx%d" % (width, height))
heading = Label(text="House Price Prediction System", font=("Arial Black", 22))
heading.pack()

#creating different tabs in the window
tab_parent = Notebook(window)
tab1 = Frame(tab_parent)
tab2 = Frame(tab_parent)
tab3 = Frame(tab_parent)
tab_parent.add(tab1, text='Your Interest City Search')
tab_parent.add(tab2, text="Know the Price")
tab_parent.add(tab3, text="Research")
#creating a list of countries name
countries = list(df.Country.unique())

#function to pick county name as per country selected
def pick_county(e):
    if cm1.get()=="England":
        data = list(df.county[df.Country=='England'].values)
        cm2.config(value=data)
    elif cm1.get()=="Wales":
        data = list(df.county[df.Country=='Wales'].values)
        cm2.config(value=data)
    elif cm1.get()=="Scotland":
        data = list(df.county[df.Country=='Scotland'].values)
        cm2.config(value=data)
    elif cm1.get()=="Northern Ireland":
        data = list(df.county[df.Country=='Northern Ireland'].values)
        cm2.config(value=data)

#function to pick city name as per county name selected
def pick_city(e):
    data = list(df.City[df.county == cm2.get()].values)
    cm3.config(value=data)

    

#getting the co-ordinates of the location
def location(location):
    geolocator = Nominatim(user_agent="uk")
    address = str(location +", United Kingdom")
    location = geocoder.arcgis(address)
    latlang = location.latlng
    return latlang

#class to plot the graphs using clustering
class interests:
    def graph(self):
        self.loc = cm3.get()    
        self.df_final = main_function(self.loc)
        latlang =location(self.loc)
        lbl55= Label(tab1, text="Most Common Venues", font=("Helvetica", 12,))
        lbl55.place(x=50, y=100)
        map_widget =tkintermapview.TkinterMapView(tab1,width=500,height=400,corner_radius=0)
        map_widget.set_position(latlang[0],latlang[1])
        map_widget.set_zoom(11)
        map_widget.place(x=50,y=120)   
        for lat,long,name,cluster in zip(self.df_final['Latitude'], self.df_final['Longitude'], self.df_final['neighborhood'], self.df_final['Cluster Labels']):
            if cluster == 0 :
                map_widget.set_marker(lat,long, text=name,marker_color_circle='red',marker_color_outside='red')
            elif cluster == 1 :
                map_widget.set_marker(lat,long, text=name,marker_color_circle='blue',marker_color_outside='blue')
            elif cluster == 2 :
                map_widget.set_marker(lat,long, text=name,marker_color_circle='green',marker_color_outside='green')
            elif cluster == 3 :
                map_widget.set_marker(lat,long, text=name, marker_color_circle='purple',marker_color_outside='purple')
        lbl8 = Label(tab1, text="Most Common Venues :",font=('bold'))
        lbl8.place(x=600, y=150)
        lbl9 = Label(tab1, text="Red :  {}".format(most_venues(self.df_final[self.df_final['Cluster Labels']==0])),foreground='red')
        lbl9.place(x=600, y=200)
        lbl10 = Label(tab1, text="Blue :  {} ".format(most_venues(self.df_final[self.df_final['Cluster Labels']==1])),foreground='blue')
        lbl10.place(x=600, y=230)
        lbl11 = Label(tab1, text="Green :  {} ".format(most_venues(self.df_final[self.df_final['Cluster Labels']==2])),foreground='green')
        lbl11.place(x=600, y=260)
        lbl56 = Label(tab1, text="Purple :  {} ".format(most_venues(self.df_final[self.df_final['Cluster Labels']==3])),foreground='purple')
        lbl56.place(x=600, y=290)   
        
        lbl57= Label(tab1, text="Average House Prices", font=("Helvetica", 12))
        lbl57.place(x=800, y=100)
        map_widget1 =tkintermapview.TkinterMapView(tab1,width=500,height=400,corner_radius=0)
        map_widget1.set_position(latlang[0],latlang[1])
        map_widget1.set_zoom(11)
        map_widget1.place(x=800,y=120)
        temp_loc = df_price[df_price.city1==self.loc]
        max_val = temp_loc.Avg_asking_price.max()
        min_val =temp_loc.Avg_asking_price.min()
        labels=['low','medium','high']
        bins = np.linspace(min_val,max_val,4)
        temp_loc['bins'] = pd.cut(temp_loc['Avg_asking_price'], bins=bins, labels=labels, include_lowest=True)
        for lat,long,post,bins in zip(temp_loc['Latitude'], temp_loc['Longitude'],temp_loc['Postcode'], temp_loc['bins']):
            if bins == "low" :
                map_widget1.set_marker(lat,long, text=post,marker_color_circle='pink',marker_color_outside='pink')
            elif bins == "medium" :
                map_widget1.set_marker(lat,long, text=post,marker_color_circle='orange',marker_color_outside='orange')
            elif bins == "high" :
                map_widget1.set_marker(lat,long, text=post,marker_color_circle='cyan',marker_color_outside='cyan')
        lbl58 = Label(tab1, text="House Prices:", font='bold')
        lbl58.place(x=600, y=350)
        temp_low = temp_loc[temp_loc.bins=='low']
        min_low = temp_low.Avg_asking_price.min()
        max_low = temp_low.Avg_asking_price.max()
        lbl59 = Label(tab1, text="Pink :£{}-£{}".format(min_low,max_low),foreground='pink')
        lbl59.place(x=600, y=370)
        temp_med = temp_loc[temp_loc.bins=='medium']
        min_med = temp_med.Avg_asking_price.min()
        max_med = temp_med.Avg_asking_price.max()
        lbl60 = Label(tab1, text="Orange :£{}-£{}".format(min_med,max_med),foreground='orange')
        lbl60.place(x=600, y=400)
        temp_high = temp_loc[temp_loc.bins=='low']
        min_high = temp_high.Avg_asking_price.min()
        max_high = temp_high.Avg_asking_price.max()
        lbl61 = Label(tab1, text="Cyan :£{}-£{}".format(min_high,max_high),foreground='cyan')
        lbl61.place(x=600, y=430)
                    
    
    #function to work when know more is clicked
    def know_more(self):
        new_win = Toplevel(tab1)
        new_win.geometry("%dx%d" % (width, height))
        lbl4 = Label(new_win, text="Most Common Venues in {} :".format(self.loc), font=("Helvetica", 15))
        lbl4.place(x=110, y=20)
        table = Treeview(new_win)
        table['columns'] = ('Neighborhood', 'Latitude','Longitude','1st Most Common Venue',
                            '2nd Most Common Venue','3rd Most Common Venue','4th Most Common Venue','5th Most Common Venue')
        table.column("#0", width=0, stretch=NO)
        table.column("Neighborhood", anchor=CENTER, width=100)
        table.column("Latitude", anchor=CENTER, width=150)
        table.column("Longitude", anchor=CENTER, width=150)
        table.column("1st Most Common Venue", anchor=CENTER, width=150)
        table.column("2nd Most Common Venue", anchor=CENTER, width=150)
        table.column("3rd Most Common Venue", anchor=CENTER, width=150)
        table.column("4th Most Common Venue", anchor=CENTER, width=150)
        table.column("5th Most Common Venue", anchor=CENTER, width=150)
        table.heading("#0", text="", anchor=CENTER)
        table.heading("Neighborhood", text="Neighborhood", anchor=CENTER)
        table.heading("Latitude", text="Latitude", anchor=CENTER)
        table.heading("Longitude", text="Longitude", anchor=CENTER)
        table.heading("1st Most Common Venue", text="1st Most Common Venue", anchor=CENTER)
        table.heading("2nd Most Common Venue", text="2nd Most Common Venue", anchor=CENTER)
        table.heading("3rd Most Common Venue", text="3rd Most Common Venue", anchor=CENTER)
        table.heading("4th Most Common Venue", text="4th Most Common Venue", anchor=CENTER)
        table.heading("5th Most Common Venue", text="5th Most Common Venue", anchor=CENTER)
        output_data = []
        for i in range(len(self.df_final)):
            output_data.append((self.df_final.iloc[i,0], self.df_final.iloc[i,1],self.df_final.iloc[i,2],self.df_final.iloc[i,4],
                               self.df_final.iloc[i,5],self.df_final.iloc[i,6],self.df_final.iloc[i,7],self.df_final.iloc[i,8]))
        for out in output_data:
            table.insert(parent='', index='end', text='', values=out)
        table.place(x=120,y=50)
        lbl5 = Label(new_win, text="House Prices per bedroom count in {} :".format(self.loc), font=("Helvetica", 15))
        lbl5.place(x=110, y=290)
        df_beds = get_bed_prices1(self.loc)
        if len(df_beds)==0:
            lbl7 = Label(new_win, text="Insufficient data for  {} Present !!! Sorry!!! :".format(self.loc), font=("Helvetica", 15))
            lbl7.place(x=120, y=350)
        else:
            figure1 = plt.Figure(figsize=(8, 4))
            ax1 = figure1.add_subplot(111)
            plot1 = FigureCanvasTkAgg(figure1, new_win)
            dict_data = {'2 Bedrooms':df_beds.price[df_beds.bedrooms==2],'3:Bedrooms':df_beds.price[df_beds.bedrooms==3],
                         '4 Bedrooms':df_beds.price[df_beds.bedrooms==4],'5 Bedrooms':df_beds.price[df_beds.bedrooms==5]}
            ax1.boxplot(dict_data.values())
            ax1.set_xticklabels(dict_data.keys())
            plot1.get_tk_widget().place(x=120, y=320)
            temp2 =df_beds[df_beds.bedrooms==2]
            temp3 =df_beds[df_beds.bedrooms==3]
            temp4 =df_beds[df_beds.bedrooms==4]
            temp5 =df_beds[df_beds.bedrooms==5]
            lbl13 = Label(new_win, text="Average Price of 2 Bedrooms:  £{:.2f}".format(temp2.price.mean()),
                                                                                                     font=("Helvetica", 10))
            lbl13.place(x=800, y=350)
            lbl13 = Label(new_win, text="Average Price of 3 Bedrooms:  £{:.2f}".format(temp3.price.mean()),
                                                                                                     font=("Helvetica", 10))
            lbl13.place(x=800, y=375)
            lbl13 = Label(new_win, text="Average Price of 4 Bedrooms:  £{:.2f}".format(temp4.price.mean()),
                                                                                                     font=("Helvetica", 10))
            lbl13.place(x=800, y=400)
            lbl13 = Label(new_win, text="Average Price of 5 Bedrooms:  £{:.2f}".format(temp5.price.mean())
                                                                                                     , font=("Helvetica", 10))
            lbl13.place(x=800, y=425)
            lbl6 = Label(new_win, text="Crimes in {} :".format(self.loc), font=("Helvetica", 15))
            lbl6.place(x=120, y=620)
        df_crime = get_crime_data1(self.loc)
        if len(df_crime) == 0:
            lbl17 = Label(new_win, text="Crimes in 12 months: No Sufficient Data Present! Sorry!!!", font=("Helvetica", 10))
            lbl17.place(x=350, y=625)
            lbl19 = Label(new_win, text="Crime Rating: No Sufficient Data Present! Sorry!!!", font=("Helvetica", 10))
            lbl19.place(x=700, y=625)

        else:
            mean_crimes = df_crime['1'].mean()
            lbl17 = Label(new_win, text="Crimes in 12 months: {:.2f}".format(mean_crimes), font=("Helvetica", 10))
            lbl17.place(x=350, y=625)
            crime_rating = df_crime['3'].mode()[0]
            lbl19 = Label(new_win, text="Crime Rating: {}".format(crime_rating), font=("Helvetica", 10))
            lbl19.place(x=700, y=625)
        b5 = Button(new_win, text="EXIT", command =new_win.destroy)
        b5.place(x=1200,y=550)

# function to pick county name as per country selected
def pcounty1(e):
    if cm7.get()=='England':
        data = list(df_reg.County[df_reg.Country=='England'].unique())
        data.sort()
        cm8.config(value=data)

    elif cm7.get()=="Wales":
        data = list(df_reg.County[df_reg.Country=='Wales'].unique())
        data.sort()
        cm8.config(value=data)

#function to pick district name as per county selected
def pdistrict1(e):
    data = list(df_reg.District[df_reg.County == cm8.get()].unique())
    data.sort()
    cm9.config(value=data)    

#function to pickcity name as per district selected    
def pcity1(e):
    data =list(df_reg.Town_City[df_reg.District == cm9.get()].unique())
    data.sort()
    cm10.config(value=data)
    
#function to pick areacode as per city selected
def pareacode1(e):
    data =list(df_reg.areacode[df_reg.Town_City == cm10.get()].unique())
    data.sort()
    cm11.config(value=data)


#function to calculate the property prices using XGBoost machine learning model
def calc_property_price():
    rank = df_city.Rank[df_city.city==cm8.get()]
    c_rank =rank.values[0]
    header = df_dummy.columns
    df_prediction = pd.DataFrame(columns=header)
    df_prediction = df_prediction.append(pd.Series(0, index=df_prediction.columns), ignore_index=True)
    areacode = cm11.get()
    if (cm12.get() =="Detached"):
        property_type='D'
    elif (cm12.get()=="Semi-Detached"):
        property_type='S'
    elif cm12.get() == "Terraced":
        property_type ='T'
    elif cm12.get() == "Flats/Maisonettes":
        property_type ='F'
    elif cm12.get() == "Other":
        property_type ='O'
    if cm13.get()=="Newly Built":
        property_age =1
    elif cm13.get()=="Established":
        property_age =0
    if cm14.get()=="Freehold":
        property_tenure=0
    elif cm14.get()=="Leasehold":
        property_tenure =1
    if cm15.get()=="single residential property sold":
        property_ppd =0
    elif cm15.get()=="buy-to-lets/others":
        property_ppd =1
    property_epc = cm16.get()
    property_year = cm17.get()
    fl_area = total_floor_area_e.get()
    energy_tar = cm18.get()        
    df_prediction['Old/New']=property_age
    df_prediction['Property_Type_{}'.format(property_type)]=1
    df_prediction['Duration']=property_tenure
    df_prediction['PPD\xa0Category Type']=property_ppd
    df_prediction['construction_age_band_ {}'.format(property_year)]=1
    df_prediction['current_energy_rating_{}'.format(property_epc)]=1
    df_prediction['total_floor_area']=fl_area
    df_prediction['energy_tariff_{}'.format(energy_tar)]=1
    df_prediction['areacode_{}'.format(areacode)]=1
    df_prediction['county_rank']=c_rank
    preds = model_xgboost.predict(df_prediction)
    lbl62 = Label(tab2, text="Estimate Property Price :£{:.2f}".format(preds[0]),font=("Arial Black", 12))
    lbl62.place(x=750, y=250)  


#function to calculate the prices of development and display profit percentage
def calc_profit():
    url ='https://api.propertydata.co.uk/development-calculator?key=L1SGXGR1LQ&postcode={}&purchase_price={}&sqft_pre_development={}&sqft_post_development={}&project_type={}&finish_quality={}'.format(postcode_e1.get(),ini_e2.get(),sqp_e3.get(),sqa_e4.get(),dev_cm5.get(),fin_cm6.get())
    response = requests.get(url)
    data = response.json()
    lbl29=Label(tab2, text="Stamp Duty: £{}".format(data['results']['stampDuty']['value']), font=("Arial Black", 11))
    lbl29.place(x=850,y=300)
    lbl30=Label(tab2, text="Legal fees: £{}".format(data['results']['legals']['value']), font=("Arial Black", 11))
    lbl30.place(x=850,y=320)
    lbl31=Label(tab2, text="Architects fees: £{}".format(data['results']['architects']['value']), font=("Arial Black", 11))
    lbl31.place(x=850,y=340)
    lbl32=Label(tab2, text="Construction Cost: £{}".format(data['results']['construction']['value']), font=("Arial Black", 11))
    lbl32.place(x=850,y=360)
    lbl33=Label(tab2, text="subTotal Costs: £{}".format(data['results']['subTotalCosts']['value']), font=("Arial Black", 11))
    lbl33.place(x=850,y=380)
    lbl34=Label(tab2, text="Contingency Amount: £{}".format(data['results']['contingency']['value']), font=("Arial Black", 11))
    lbl34.place(x=850,y=400)
    lbl35=Label(tab2, text="finance costs: £{}".format(data['results']['finance']['value']), font=("Arial Black", 11))
    lbl35.place(x=850,y=420)
    lbl36=Label(tab2, text="total Costs: £{}".format(data['results']['totalCosts']['value']), font=("Arial Black", 11))
    lbl36.place(x=850,y=440)
    lbl37=Label(tab2, text="Average asking price:(persqf) £{}".format(data['results']['valuePsf']['value']), font=("Arial Black", 11))
    lbl37.place(x=850,y=460)
    lbl38=Label(tab2, text="Total Value: £{}".format(data['results']['totalValue']['value']), font=("Arial Black", 11))
    lbl38.place(x=850,y=480)
    lbl39=Label(tab2, text="Profit: £{}".format(data['results']['profit']['value']), font=("Arial Black", 11))
    lbl39.place(x=850,y=500)
    lbl40=Label(tab2, text="Profit Percentage: {}".format(data['results']['profitPc']['value']), font=("Arial Black", 12))
    lbl40.place(x=850,y=540)

#function to forecast HPI and display on frame 3
def HPI_pred(temp_input,x_input):
    lst_output=[]
    n_steps=30
    i=0
    while(i<30):

        if(len(temp_input)>30):
            x_input=np.array(temp_input[1:])
            x_input=x_input.reshape(1,-1)
            x_input = x_input.reshape((1, n_steps, 1))
            yhat = model_hpi.predict(x_input, verbose=0)
            temp_input.extend(yhat[0].tolist())
            temp_input=temp_input[1:]
            lst_output.extend(yhat.tolist())
            i=i+1
        else:
            x_input = x_input.reshape((1, n_steps,1))
            yhat = model_hpi.predict(x_input, verbose=0)
            temp_input.extend(yhat[0].tolist())
            lst_output.extend(yhat.tolist())
            i=i+1
    return lst_output


#loading models
model_hpi = load_model(r'C:\Users\hp\dissertation\Model\HPI_model.H5')
model_xgboost = joblib.load(r'C:\Users\hp\dissertation\Model\xgboost_model1.sav')
    
    
#calling class modules
inter = interests()

#widgets for tab1
lbl1 = Label(tab1, text="Please choose the country :", font=("Helvetica", 15))
lbl1.place(x=20, y=50)
cm1 = Combobox(tab1,values=countries)
cm1.place(x=270, y=53)
cm1.bind("<<ComboboxSelected>>",pick_county)
lbl2 = Label(tab1, text="County :", font=("Helvetlstica", 15))
lbl2.place(x=450, y=50)
cm2 = Combobox(tab1,values=[" "])
cm2.place(x=540,y=53)
cm2.bind("<<ComboboxSelected>>",pick_city)
lbl3 = Label(tab1, text="City :", font=("Helvetica", 15))
lbl3.place(x=750, y=50)
cm3 = Combobox(tab1,values=[" "])
cm3.place(x=810,y=53)
b1 = Button(tab1, text="Go!",command =inter.graph)
b1.place(x=1000,y=53)
lb =Label(tab1,text=cm3.get())
lb.place(x=1200,y=500)
b2 = Button(tab1, text="Know More!",command=inter.know_more)
b2.place(x=600,y=500)
b3 = Button(tab1, text="EXIT", command =window.destroy)
b3.place(x=1200,y=550)

#tab2 widgets
lbl20 =Label(tab2, text="Please enter the following details to Know the House Price :", font=("Arial Black", 12))
lbl20.place(x=50, y=10)
c_name =['England','Wales']
lbl45 = Label(tab2, text ="Select the Country :")
lbl45.place(x=60,y=40)
cm7 = Combobox(tab2,values=c_name)
cm7.place(x=290,y=40)
cm7.bind("<<ComboboxSelected>>",pcounty1)
lbl46 =Label(tab2, text ="Select the County :")
lbl46.place(x=60,y=70)
cm8 = Combobox(tab2,values=[" "])
cm8.place(x=290,y=70)
cm8.bind("<<ComboboxSelected>>",pdistrict1)
lbl47 = Label(tab2,text="Select the District :")
lbl47.place(x=60,y=100)
cm9 =Combobox(tab2,values= [" "])
cm9.place(x=290,y=100)
cm9.bind("<<ComboboxSelected>>",pcity1)
lbl48 = Label(tab2,text="Select the Town/City :")
lbl48.place(x=60,y=130)
cm10 =Combobox(tab2,values= [" "])
cm10.place(x=290,y=130)
cm10.bind("<<ComboboxSelected>>",pareacode1)
lbl49 = Label(tab2,text="Select the Post code :")
lbl49.place(x=60,y=160)
cm11 =Combobox(tab2,values= [" "])
cm11.place(x=290,y=160)
lbl50 = Label(tab2,text="Select the Property Type:")
lbl50.place(x=60,y=190)
prop_type = ['Detached', 'Semi-Detached', 'Terraced','Flats/Maisonettes','Other']
cm12 = Combobox(tab2,values=prop_type)
cm12.place(x=290,y=190)
lbl51 = Label(tab2,text="Select the Age of Property:")
lbl51.place(x=60,y=220)
prop_age = ['Newly Built','Established']
cm13 = Combobox(tab2,values=prop_age)
cm13.place(x=290,y=220)
lbl52 = Label(tab2,text="Select the Tenure of Property:")
lbl52.place(x=60,y=250)
prop_tenure = ['Freehold','Leasehold']
cm14 = Combobox(tab2,values=prop_tenure)
cm14.place(x=290,y=250)
lbl52 = Label(tab2,text="Select the type of Price Paid transaction:")
lbl52.place(x=60,y=280)
prop_ppd = ['single residential property sold','buy-to-lets/others']
cm15 = Combobox(tab2,values=prop_ppd)
cm15.place(x=290,y=280)
lbl53 = Label(tab2,text="Select Energy Ratings:")
lbl53.place(x=600,y=40)
prop_epc = ['A','B','C','D','E','F','G','0']
cm16 = Combobox(tab2,values=prop_epc)
cm16.place(x=790,y=40)
lbl53 = Label(tab2,text="Select Year Built:")
lbl53.place(x=600,y=70)
prop_year = ['1950-1966','1900-1929','1930-1949','1967-1975','before 1900','1983-1990','1976-1982','1996-2002','1991-1995',
 '2003-2006','2007-2011','2007 onwards','2012 onwards']
prop_year.sort()
cm17 = Combobox(tab2,values=prop_year)
cm17.place(x=790,y=70)
lbl54 = Label(tab2,text="Enter Total Floor Area:")
lbl54.place(x=600,y=100)
total_floor_area_e = Entry(tab2,width=23)
total_floor_area_e.place(x = 790, y = 100) 
lbl55 = Label(tab2,text="Select Energy Tariff:")
lbl55.place(x=600,y=130)
prop_tariff =['Single','Unknown','dual','off-peak 7 hour','standard tariff',
              'off-peak 10 hour','dual (24 hour)','off-peak 18 hour','24 hour']
prop_tariff.sort()
cm18 = Combobox(tab2,values=prop_tariff)
cm18.place(x=790,y=130)
b8 = Button(tab2, text="Calculate Price!", command=calc_property_price)
b8.place(x=700,y=200)
lbl22 =Label(tab2, text="Please enter the following details to Know the Profit on Property :", font=("Arial Black", 12))
lbl22.place(x=50, y=320)
lbl23 =Label(tab2, text="Full UK PostCode :", font=("Helvetica", 10))
lbl23.place(x=50, y=370)
postcode_e1 = Entry(tab2,width=23)
postcode_e1.place(x = 500, y = 370) 
lbl24 =Label(tab2, text="Initial purchase price(in pounds) :", font=("Helvetica", 10))
lbl24.place(x=50, y=400)
ini_e2 = Entry(tab2,width=23)
ini_e2.place(x = 500, y = 400) 
lbl25 =Label(tab2, text="The size of the property before development(in square feet) :", font=("Helvetica", 10))
lbl25.place(x=50, y=430)
sqp_e3 = Entry(tab2,width=23)
sqp_e3.place(x = 500, y = 430) 
lbl26 =Label(tab2, text="The size of the property after development(in square feet) :", font=("Helvetica", 10))
lbl26.place(x=50, y=460)
sqa_e4 = Entry(tab2,width=23)
sqa_e4.place(x = 500, y = 460) 
lbl27 =Label(tab2, text="Type of development project (refurbish / demolition) :", font=("Helvetica", 10))
lbl27.place(x=50, y=490)
data_dev =['refurbish','demolition']
dev_cm5=Combobox(tab2,values=data_dev)
dev_cm5.place(x = 500, y = 490) 
lbl28 =Label(tab2, text="Interior finish quality (premium / medium / basic) :", font=("Helvetica", 10))
lbl28.place(x=50, y=520)
data_fin =['premium','medium','basic'] 
fin_cm6=Combobox(tab2,values=data_fin)
fin_cm6.place(x = 500, y = 520) 
b4 = Button(tab2, text="Check!", command =calc_profit)
b4.place(x=550,y=550)
b6 = Button(tab2, text="EXIT", command =window.destroy)
b6.place(x=1200,y=550)

#tab3 widgets
#plotting prediction
df_mod = df_model[df_model.Region_Name=='United Kingdom']
lbl41 =Label(tab3, text="House Price Index Prediction :", font=("Helvetica", 10))
lbl41.place(x=50, y=60)
figure2 = plt.Figure(figsize=(7,3))
ax2 = figure2.add_subplot(111)
plot2 = FigureCanvasTkAgg(figure2, tab3)
data1=df_mod[['Index']].values
scaler = MinMaxScaler(feature_range=(0,1))
data1 = scaler.fit_transform(np.array(data1).reshape(-1,1))
x_input =data1[-30:]
temp_input=list(x_input)
temp_input=temp_input[0].tolist()
preds = HPI_pred(temp_input,x_input)
predicted_value =scaler.inverse_transform(preds).tolist()
actual_value =(df_mod[['Index']].values).tolist()[-300:]
actual_value.extend(predicted_value)
day_new=np.arange(1,331)
ax2.plot(day_new,actual_value)
plot2.get_tk_widget().place(x=50, y=100)

#plotting prices as per land registry
df_uk = df_land[df_land.RegionName=='United Kingdom']
df_scot = df_land[df_land.RegionName=='Scotland']
df_eng = df_land[df_land.RegionName=='England']
df_wales= df_land[df_land.RegionName=='Wales']
df_NI = df_land[df_land.RegionName=='Northern Ireland']
lbl42 =Label(tab3, text="Average House Prices Country Wise :", font=("Helvetica", 10))
lbl42.place(x=650, y=60)
figure3 = plt.Figure(figsize=(7,3))
ax3 = figure3.add_subplot(111)
plot3 = FigureCanvasTkAgg(figure3, tab3)
sns.lineplot(data = df_scot, x='Date',y='AveragePrice', label='Scotland',ax=ax3)
sns.lineplot(data = df_eng, x='Date',y='AveragePrice', label='England',ax=ax3)
sns.lineplot(data = df_wales, x='Date',y='AveragePrice', label='Wales',ax=ax3)
sns.lineplot(data = df_NI, x='Date',y='AveragePrice', label='Northern Ireland',ax=ax3)
plot3.get_tk_widget().place(x=650, y=100)

#plot for average house price per region
property_type =['Detached', 'Semi-Detached', 'Terraced','Flat']
uk_property = [df_uk['DetachedPrice'].mean(),df_uk['SemiDetachedPrice'].mean(),
               df_uk['TerracedPrice'].mean(),df_uk['FlatPrice'].mean()]
lbl43 =Label(tab3, text="Average House Prices as per Property type in United Kingdom :", font=("Helvetica", 10))
lbl43.place(x=50, y=350)
figure4 = plt.Figure(figsize=(7,3))
ax4 = figure4.add_subplot(111)
plot4 = FigureCanvasTkAgg(figure4, tab3)
ax4.bar(property_type,uk_property)
plot4.get_tk_widget().place(x=50, y=380)

#plotting average sales volume
country_name = ['ENGLAND', 'SCOTLAND','WALES','NORTHERN IRELAND']
sales_lst = [df_eng['SalesVolume'].mean(),df_scot['SalesVolume'].mean(),df_wales['SalesVolume'].mean(),
             df_NI['SalesVolume'].mean()]
lbl44 =Label(tab3, text="Average Sales Volume of Properties as per Country:", font=("Helvetica", 10))
lbl44.place(x=650, y=350)
figure5 = plt.Figure(figsize=(7,3))
ax5 = figure5.add_subplot(111)
plot5 = FigureCanvasTkAgg(figure5, tab3)
ax5.bar(country_name,sales_lst)
plot5.get_tk_widget().place(x=650, y=380)
b7 = Button(tab3, text="EXIT", command =window.destroy)
b7.place(x=1200,y=550)

tab_parent.pack(expand=1, fill="both")
window.mainloop()